In [1]:
from commonfunctions import *
import joblib
import imageio as iio
import cv2
from skimage import filters
from skimage.color import rgb2gray  # only needed for incorrectly saved images
from skimage.measure import regionprops
from skimage import data, color, feature,morphology
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from feature_extraction import *
from pre_processing import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC,SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
%load_ext autoreload
%autoreload 2
%matplotlib inline


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# input image dimensions
img_rows, img_cols = 32, 32
# The images are RGB.
img_channels = 3
nb_classes = 9

datagen = ImageDataGenerator()
data = datagen.flow_from_directory('ACdata_base',
                                    target_size=(500, 500),
                                    batch_size=73139,
                                    class_mode='sparse',
                                   color_mode='grayscale',
                                    shuffle=True,
                                    seed=42 )
X , y = data.next()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=11);
#X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=11);
X_train.shape

In [ ]:
print(type(X_test[0]))

In [ ]:
pre_processed_train=[pre_process(patch) for patch in X_train] # de bet7awel 2el training set le binary fa law feature sha8ala 3ala binary hanesta5dem da
cropped_images=[crop_image(patch) for patch in pre_processed_train]
resized_cropped_images=[cv2.resize(np.array(cropped, dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA) for cropped in cropped_images]


In [ ]:
hough=np.array([ get_max_theta(img) for img in X_train])

In [ ]:
GABOR=np.array([gabor_filter(img.reshape(img.shape[0],img.shape[1])) for img in X_train]).reshape(-1,32)

In [ ]:
GLCM=np.array([GLCM_features(img) for img in resized_cropped_images]).reshape(-1,16)

In [ ]:
lvl=np.array([LVL(img) for img in X_train])

In [ ]:
LPQ=[lpq(img.reshape(img.shape[0],img.shape[1]),winSize=10) for img in resized_cropped_images]

In [ ]:
LPQ=np.array(LPQ)
print(LPQ.shape)

In [ ]:
hu_moments=np.array([hu_moments_func(img)for img in resized_cropped_images])

In [ ]:
print(hu_moments.shape)

In [ ]:
lbp=np.array([LBP(img,numPoints=24,radius=3,method="uniform",window=500) for img in X_train])

In [ ]:
ones=np.array([count_ones(img) for img in resized_cropped_images]).reshape(-1,1)

In [ ]:
corners=np.array([get_corners(img) for img in X_train]).reshape(-1,1)

In [ ]:
max_v=np.array([cv2.resize(get_max_vertical(patch), (50,1), interpolation = cv2.INTER_AREA) for patch in resized_cropped_images]).reshape(-1,50)

In [ ]:
diagonal=np.array([project_image_diagonal(patch) for patch in X_train]).reshape(-1,50)

In [ ]:
center_of_masses=np.array([center_of_mass(img) for img in resized_cropped_images]).reshape(-1,1)

In [ ]:
values = []
for gray in X_train:
    print(gray.shape)
    values.append(HVSL(np.uint8(gray)))
print(np.asarray(values).reshape(-1,1))
values=np.asarray(values).reshape(-1,1)

In [ ]:
from skimage import data, color, feature
patches_hog = np.array([feature.hog(patch,cells_per_block=(3, 3),pixels_per_cell=(150, 150), orientations=7) for patch in X_train])

In [ ]:
print(patches_hog.shape)

In [ ]:

#fel paper kan 3ayezna nda5al skeleton image 3ala histogram of gradienta fa garabna neda5alo 3al hog
patches_hog_skeleton = np.array([feature.hog(morphology.skeletonize(patch).astype(int),cells_per_block=(3, 3),pixels_per_cell=(150, 150), orientations=6) for patch in resized_cropped_images])

In [ ]:
base_line_feature2=np.array([HPP_Skeletonize(patch) for patch in resized_cropped_images])

In [ ]:
base_line_feature=np.array([cv2.resize(HPP(patch), (50,1), interpolation = cv2.INTER_AREA) for patch in resized_cropped_images]).reshape(-1,50)

In [ ]:
combined_array=np.concatenate((np.array(values).reshape(-1,1),np.array(center_of_masses).reshape(-1,1),base_line_feature2.reshape(-1,1),ones),axis=-1)

In [ ]:
print(base_line_feature.shape)

In [ ]:
combined_hog=np.concatenate((patches_hog,patches_hog_skeleton,combined_array,GLCM,lbp,GABOR),axis=-1)
#combined_hog=LPQ

In [ ]:
mean=np.mean(combined_hog)
std=np.std(combined_hog)

In [ ]:
combined_hog=(combined_hog-mean)/std

In [ ]:
print(combined_hog.shape)

In [ ]:
pca = PCA(n_components=100)
combined_hog=pca.fit_transform(combined_hog)
filename = 'pca_model.sav'
joblib.dump(pca, filename)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
model = Sequential()
model.add(Dense(50, input_dim=100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25,  activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9,activation='softmax'))
model.compile(optimizer='adam', metrics=['accuracy'],loss='categorical_crossentropy')
print(combined_hog.shape)

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="weights2.best.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
C = [checkpoint]

In [ ]:
model.fit(combined_hog,dummy_y, epochs=200, batch_size=100,callbacks=C, validation_split=0.1)

In [ ]:


print('gaussian',cross_val_score(GaussianNB(), combined_hog, y_train))

grid = GridSearchCV(LinearSVC(), {'C': [1.0, 2.0, 4.0, 8.0]})
grid.fit(combined_hog, y_train)
print('svm',grid.best_score_)
# bensave 2el model 3ashan mane3melsh train kol shwaya
filename = 'final_svm.sav'
joblib.dump(grid, filename)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = KNeighborsClassifier(n_neighbors=11)
classifier.fit(combined_hog, y_train)
print('gaussian',cross_val_score(classifier, combined_hog, y_train))

In [ ]:
from sklearn.neural_network import MLPClassifier
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(50, 20), random_state=1,    early_stopping=True)
clf = MLPClassifier(hidden_layer_sizes=(50, 25))
print('gaussian',np.mean(cross_val_score(clf, combined_hog, y_train)))
clf.fit(combined_hog, y_train)
filename = 'final_nn.sav'
joblib.dump(clf, filename)

In [ ]:
clf_nn = MLPClassifier(hidden_layer_sizes=(50, 25))
clf_for=RandomForestClassifier()
clf_svm = GridSearchCV(SVC(kernel='linear',probability=True), {'C': [1.0, 2.0, 4.0, 8.0]})
eclf2 = VotingClassifier(estimators=[
         ('nn', clf_nn),('svm',clf_svm)],
         voting='soft',
         flatten_transform=True)
print('gaussian',np.mean(cross_val_score(eclf2, combined_hog, y_train)))
eclf2.fit(combined_hog, y_train)
filename = 'final_voting.sav'
joblib.dump(eclf2, filename)

In [ ]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4=

In [ ]:

model=joblib.load('final_voting.sav')
model_pca=joblib.load('pca_model.sav')


In [ ]:
def predict(test_image):
    start=time.time()
    binarized_image=cv2.resize(np.array(crop_image(pre_process(test_image)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA)
    values=HVSL(np.uint8(test_image))
    center_of_masses=center_of_mass(binarized_image)
    base_line_feature2=HPP_Skeletonize(binarized_image)
    ones=count_ones(binarized_image)
    combined_array=np.array([values,center_of_masses,base_line_feature2,ones]).reshape(1,-1)
    GLCM=np.array(GLCM_features(binarized_image)).reshape(1,-1)
    GABOR=gabor_filter(test_image.reshape(test_image.shape[0],test_image.shape[1])).reshape(1,-1)
    lbp=LBP(test_image,numPoints=24,radius=3,method="uniform",window=500).reshape(1,-1)
    patches_hog_skeleton=feature.hog(morphology.skeletonize(binarized_image).astype(int),cells_per_block=(3, 3),pixels_per_cell=(150, 150), orientations=6).reshape(1,-1)
    patches_hog=feature.hog(test_image,cells_per_block=(3, 3),pixels_per_cell=(150, 150), orientations=7).reshape(1,-1)
    feature_vector=np.concatenate((patches_hog.reshape(1,-1),patches_hog_skeleton.reshape(1,-1),combined_array.reshape(1,-1),GLCM.reshape(1,-1),lbp,GABOR),axis=-1).reshape(1,-1)
    #print(feature_vector.shape)
    feature_vector=(feature_vector-mean)/std
    pca_output=model_pca.transform(feature_vector)
    prediction=model.predict(pca_output)
    end=time.time()
    #print(prediction)
    print(end-start)
    return prediction
    

In [ ]:
predictions=[predict(img) for img in X_test]

In [ ]:
correct=(np.array(predictions).reshape(-1).astype("int")==y_test)
print(np.array(predictions).shape)
print(correct.shape)
print("accuracy",sum(correct)/correct.shape[0])

In [ ]:
def get_max_vote(img):
    model_HVSL=joblib.load('HVSL.sav')
    model_HOG=joblib.load('hog.sav')
    model_hist=joblib.load('HIST.sav')
    model_HPP=joblib.load('HPP.sav')
    model_HOG_SKELETON=joblib.load('hog_skeletonize.sav')
    model_hist_skeleton=joblib.load('HIST_Skeleton.sav')
    model_center=joblib.load('center_of_mass.sav')
    model_hpp_skeletonize=joblib.load('HPP_Skeletonize.sav')
    model_max_v=joblib.load('MAX_V.sav')
    model_combined=joblib.load('combined.sav')
    model_lbp=joblib.load('LBP.sav')
    model_hog_combined=joblib.load('hog_combined_nn.sav')
    model_pca=joblib.load('pca_model.sav')
    glcm=np.array(GLCM_features(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA))).reshape(-1,12)
    #vote_max_v=model_max_v.predict(np.array([get_max_vertical(crop_image(pre_process(img)))]).reshape(1,-1))
    #combined_array=np.concatenate((max_v,np.array(values).reshape(-1,1),center_of_masses,base_line_feature2.reshape(-1,1)),axis=-1)
    lbp_feature=LBP(cv2.resize(img, (500,500), interpolation = cv2.INTER_AREA),radius=3,numPoints=24,method="uniform",window=500).reshape(1,-1)
    #vote_lbp=model_lbp.predict(lbp_feature)
    max_v=cv2.resize(np.array(get_max_vertical(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA)), dtype='uint8'), (50,1), interpolation = cv2.INTER_AREA)
    combined_array=np.array([HVSL(np.uint8(img)),center_of_mass(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA)),HPP_Skeletonize(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA)),get_corners(img),count_ones(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA))]).reshape(1, -1)
    #print(combined_array.shape)
    #vote_combined=model_combined.predict(combined_array)
    #vote_hpp_skeletonize=model_center.predict(np.array([HPP_Skeletonize(crop_image(pre_process(img)))]).reshape(1,-1))
    #vote_center=model_center.predict(np.array(center_of_mass(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA))[0]).reshape(1,-1))
    #vote_HVSL=model_HVSL.predict(np.array([HVSL(np.uint8(img))]).reshape(1,-1))
    #print(vote_HVSL)
    Gabor=gabor_filter(img.reshape(img.shape[0],img.shape[1])).reshape(1,32)
    base_line_features=cv2.resize(HPP(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA)), (50,1), interpolation = cv2.INTER_AREA).reshape(1,50)
    features_hog=feature.hog(cv2.resize(img, (64,48), interpolation = cv2.INTER_AREA), cells_per_block=(2, 2),pixels_per_cell=(16, 16), orientations=6).reshape(1,-1)
    features_hog_skeleton=feature.hog(morphology.skeletonize(cv2.resize(np.array(crop_image(pre_process(img)),dtype="uint8"), (64,48), interpolation = cv2.INTER_AREA)).astype("int"),cells_per_block=(2, 2),pixels_per_cell=(16, 16), orientations=6).reshape(1,-1)
    #vote_hog=model_HOG.predict(features_hog)
    #vote_hist=model_hist.predict(get_histogram_of_gradients(pre_process(img)).reshape(1,-1))
    #print(vote_hist)
    #vote_HPP=model_HPP.predict(HPP(cv2.resize(np.array(crop_image(pre_process(img)), dtype='uint8'), (500,500), interpolation = cv2.INTER_AREA)).reshape(1,-1))
    #print(vote_HPP)
    combine_array_hog=np.concatenate((features_hog,combined_array,lbp_feature,features_hog_skeleton,base_line_features,glcm,max_v,Gabor),axis=-1).reshape(1,-1)
    #print(pca.transform(combine_array_hog).shape)
    #print(combine_array_hog.shape)
    pca_output=model_pca.transform(combine_array_hog)
    vote_hog_combined=model_hog_combined.predict(combine_array_hog)
    #vote_HOG_SKELETON=model_HOG_SKELETON.predict(features_hog_skeleton)
    #vote_hist_skeleton=model_hist_skeleton.predict(get_histogram_of_gradients(morphology.skeletonize(np.array(pre_process(img),dtype="uint8")).astype("int")).reshape(1,-1))
    #print(vote_hist_skeleton)
    #votes=np.array([vote_HVSL,vote_hog,vote_HPP,vote_HOG_SKELETON]).reshape(-1).astype("int")
    votes=np.array([vote_hog_combined]).reshape(-1).astype("int")
    #print(votes)
    return np.bincount(votes).argmax()

In [ ]:
show_images([X_test[0]])
print("hehhhhh",get_max_vote(X_test[0]))
print("saaaa7",y_test[0])

In [ ]:
test_values=[get_max_vote(img) for img in X_test]
print(test_values)
print(y_test.astype("int"))
correct=test_values==y_test
show_images([y_test[test_values!=y_test]])
#print(np.bincount(y_test[test_values!=y_test]).argmax())
print("accuracy=",np.sum(correct)/correct.shape[0])

In [ ]:
print("accuracy=",np.sum(correct)/correct.shape[0])

In [ ]:
x=[(1,2,3,4),(5,6,7,8)]

In [ ]:
print(x)

In [ ]:
print(np.asarray(x))

In [ ]:
329/334

In [ ]:
show_images(X_test[np.array(predictions).reshape(-1).astype("int")!=y_test])
print(y_test[np.array(predictions).reshape(-1).astype("int")!=y_test])
print(np.array(predictions).reshape(-1).astype("int")[np.array(predictions).reshape(-1).astype("int")!=y_test])
